# Personal Finance Insights Dashboard
---

## Objetivo general del EDA

El objetivo principal es comprender los patrones financieros personales, observando cómo influyen factores como la edad, la educación o el empleo en indicadores clave como la deuda, el ahorro y la puntuación crediticia.


1.	Analizar la relación entre variables financieras clave, como:

	* debt_to_income_ratio: proporción entre deuda e ingresos.
	* credit_score: puntuación crediticia (300–850).
	* savings_to_income_ratio: proporción entre ahorro e ingresos.
	* region: zona geográfica de residencia.
2.	Comparar los indicadores financieros según variables demográficas y laborales:
	* Edad (age).
	* Género (gender).
	* Nivel educativo (education_level).
	* Situación laboral (employment_status).
	* Puesto de trabajo (job_title).

3.	Identificar posibles patrones o desigualdades financieras entre grupos de población, regiones o tipos de empleo.

4.	Detectar relaciones de interés entre el perfil personal y la estabilidad económica (por ejemplo, cómo varía el credit_score según la educación o la relación entre ingresos y ahorro por tipo de empleo).

---

## Visualizaciones Incluidas

* Distribuciones y boxplots de ratios financieros (DTI, Savings/Income, Credit Score).
* Comparativas por categoría (género, educación, empleo, región).
* Relaciones entre variables numéricas mediante scatter plots y correlaciones.
* Tablas y resúmenes estadísticos para observar diferencias entre grupos.
* Gráficos de barras para analizar medias y medianas de indicadores financieros por categoría.

---

## Valor del Análisis

* Permite entender el comportamiento financiero medio de diferentes perfiles de personas.
* Facilita detectar grupos con mayor riesgo o menor estabilidad económica (por ejemplo, alto DTI o bajo ahorro).
* Proporciona una base sólida para futuras fases de modelado o segmentación, en caso de ampliarse el proyecto.
* Ayuda a visualizar desigualdades financieras entre regiones, niveles educativos o tipos de empleo.

# 1.- Librerías

In [ ]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd #manejo de estructura de datos tabulares
import numpy as np # manejo de arrays y operaciones matemáticas

# libreria para crear mapas interactivos
# -----------------------------------------------------------------------
import folium
from folium.plugins import HeatMap, MarkerCluster


# librerias para visualizar datos
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud


# Libreria para acceder a funciones del sistema 
# -----------------------------------------------------------------------
import sys 
sys.path.append("../")
import src.suport as sp


# Warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Configuracion de pandas
# -----------------------------------------------------------------------
pd.options.display.max_columns = None

# Next Steps

- Profundizar en el análisis correlacional entre ratios financieros.
- Explorar la distribución del score crediticio por región y situación laboral.
- Analizar los niveles de ahorro en relación con los ingresos y la edad.
- Documentar los hallazgos visuales y las conclusiones generales del EDA.